# Сборный проект

## Введение

Выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

похоже, найден оригинал данных…

[video game sales on kaggle.com](https://www.kaggle.com/datasets/gregorut/videogamesales) 🤫

<div class="alert alert-info" role="alert">
    <p class="mb-0">В коде использую <code>display()</code>, потомучто это однозначно даёт понять что планируется вывод / печать результатов. То есть использую <code>display()</code> намеренно, хотя понимаю что можно и без него выводить данные. 👌</p>
</div>

<hr>

## Шаг 0

In [1]:
import pandas as pd
import seaborn as sns
import scipy.stats as st
import numpy as np

from IPython.core.display_functions import display


In [2]:
%config InlineBackend.figure_formats = ['svg']

In [3]:
# seaborn style via MATPLOTLIBRC
custom_params = {'figure.figsize': (15, 7),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,

                'text.color': '#EEEEEE',
                'font.sans-serif': [
                    'Helvetica',
                    'Verdana',
                    'sans-serif'
                    ]
                }
sns.set_theme(style='darkgrid', rc=custom_params)
sns.set_context("notebook", font_scale=1.37)

<hr>

## Шаг 1. Откройте файл с данными и изучите общую информацию

Загружаем данные тестового задания.

In [4]:
try:
    games_df = pd.read_csv('./datasets/games.csv')
except FileNotFoundError:
    games_df = pd.read_csv('https://code.s3.yandex.net/datasets/games.csv')
    print('games_df loaded by url')

Заменим названия столбцов (приведем к нижнему регистру);

In [5]:
games_df.columns = games_df.columns.str.lower()

In [6]:
display(
    games_df.sample(7)
)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
11759,Party Pigs: Farmyard Games,Wii,2009.0,Misc,0.07,0.00,0.00,0.00,NaN,tbd,E
8492,Curious George,GBA,2006.0,Action,0.12,0.04,0.00,0.00,65.0,tbd,E
3080,Brothers In Arms: Road to Hill 30,XB,2005.0,Shooter,0.49,0.14,0.00,0.02,88.0,8,M
6803,Disney Stitch Jam,DS,2009.0,Misc,0.07,0.00,0.16,0.01,NaN,tbd,E
9988,Cabela's North American Adventures,PS3,2010.0,Sports,0.11,0.00,0.00,0.01,NaN,tbd,T
14089,Just Dance 2017,XOne,2016.0,Misc,0.03,0.00,0.00,0.00,75.0,8,E10+
9240,Shining Soul,GBA,2002.0,Role-Playing,0.03,0.01,0.09,0.00,58.0,8.5,E


In [7]:
display(
    games_df.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


None

### Колонки с пропусками:

In [8]:
display(
    games_df.columns[games_df.isna().any()].tolist()
)

['name', 'year_of_release', 'genre', 'critic_score', 'user_score', 'rating']

⭕️ 'name' `NaN`
⭕️ 'year_of_release' `NaN`  
⭕️ 'genre' `NaN`  
⭕️ 'critic_score' `NaN`  
⭕️ 'user_score' `NaN`  
⭕️ 'rating' `NaN`  

#### `name` — название игры

In [9]:
display(
    games_df.loc[games_df['name'].isna()]
)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


In [10]:
games_df.dropna(axis=0, subset=['name'], inplace=True)

In [11]:
display(games_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


None

✅ ‘name’ `NaN`  
⭕️ 'year_of_release' `NaN`  
✅ 'genre' `NaN`  
⭕️ 'critic_score' `NaN`  
⭕️ 'user_score' `NaN`  
⭕️ 'rating' `NaN`  

In [12]:
display(
    games_df.loc[games_df['name'] == 'Madden NFL 13']
)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
507,Madden NFL 13,X360,2012.0,Sports,2.53,0.15,0.0,0.17,81.0,5.8,E
604,Madden NFL 13,PS3,2012.0,Sports,2.11,0.22,0.0,0.23,83.0,5.5,E
3986,Madden NFL 13,Wii,2012.0,Sports,0.47,0.00,0.0,0.03,NaN,7.3,E
5887,Madden NFL 13,PSV,2012.0,Sports,0.28,0.00,0.0,0.02,63.0,7.3,E
7066,Madden NFL 13,WiiU,2012.0,Sports,0.21,0.00,0.0,0.02,75.0,6.7,E
16230,Madden NFL 13,PS3,2012.0,Sports,0.00,0.01,0.0,0.00,83.0,5.5,E


In [13]:
display(
    len(games_df['name'].unique())
)

11559

#### `year_of_release` — год выпуска

In [14]:
display(
    games_df.groupby(
        by='year_of_release'
    )['name'].count()
)

year_of_release
1980.0       9
1981.0      46
1982.0      36
1983.0      17
1984.0      14
1985.0      14
1986.0      21
1987.0      16
1988.0      15
1989.0      17
1990.0      16
1991.0      41
1992.0      43
1993.0      60
1994.0     121
1995.0     219
1996.0     263
1997.0     289
1998.0     379
1999.0     338
2000.0     350
2001.0     482
2002.0     829
2003.0     775
2004.0     762
2005.0     939
2006.0    1006
2007.0    1197
2008.0    1427
2009.0    1426
2010.0    1255
2011.0    1136
2012.0     653
2013.0     544
2014.0     581
2015.0     606
2016.0     502
Name: name, dtype: int64

`genre` — жанр игры

In [15]:
display(
    games_df['genre'].unique(),
    games_df.loc[
        games_df['genre'].isna()
    ]['name'].count()
)

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

0

#### `critic_score` — оценка критиков (максимум 100)

In [16]:
display(
    games_df['critic_score'].describe()
)

count    8137.000000
mean       68.967679
std        13.938165
min        13.000000
25%        60.000000
50%        71.000000
75%        79.000000
max        98.000000
Name: critic_score, dtype: float64

In [17]:
display('Продажи игр, у которых пропущена оценка критиков',
    games_df.loc[
        games_df['critic_score'].isna()
        , ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
    ].describe(),
    'Продажи игр, у которых известна оценка критиков',
    games_df.loc[
        games_df['critic_score'].notna()
        , ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
    ].describe()
)

'Продажи игр, у которых пропущена оценка критиков'

,na_sales,eu_sales,jp_sales,other_sales
count,8576.000000,8576.000000,8576.000000,8576.000000
mean,0.178215,0.085017,0.099065,0.023535
std,0.715940,0.321036,0.343974,0.088748
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.020000,0.000000,0.000000,0.000000
75%,0.140000,0.050000,0.060000,0.020000
max,29.080000,10.950000,10.220000,2.740000


'Продажи игр, у которых известна оценка критиков'

,na_sales,eu_sales,jp_sales,other_sales
count,8137.000000,8137.000000,8137.000000,8137.000000
mean,0.352980,0.208311,0.055028,0.072435
std,0.896476,0.635622,0.265075,0.249204
min,0.000000,0.000000,0.000000,0.000000
25%,0.050000,0.010000,0.000000,0.010000
50%,0.130000,0.050000,0.000000,0.020000
75%,0.340000,0.180000,0.010000,0.060000
max,41.360000,28.960000,6.500000,10.570000


#### `platform` — платформа

In [18]:
display(
    sorted(games_df['platform'].unique()),
    len(games_df['platform'].unique()),
)

['2600',
 '3DO',
 '3DS',
 'DC',
 'DS',
 'GB',
 'GBA',
 'GC',
 'GEN',
 'GG',
 'N64',
 'NES',
 'NG',
 'PC',
 'PCFX',
 'PS',
 'PS2',
 'PS3',
 'PS4',
 'PSP',
 'PSV',
 'SAT',
 'SCD',
 'SNES',
 'TG16',
 'WS',
 'Wii',
 'WiiU',
 'X360',
 'XB',
 'XOne']

31

✅ ‘name’ `NaN`
⭕️ 'year_of_release' `NaN`
✅ 'genre' `NaN`
⭕️ 'critic_score' `NaN`
⭕️ 'user_score' `NaN`
⭕️ 'rating' `NaN`

#### `user_score` — оценка пользователей (максимум 10)

In [19]:
display(
    games_df['user_score'].describe()
)

count     10014
unique       96
top         tbd
freq       2424
Name: user_score, dtype: object

**tbd — To Be Determined.**
В интернетах пишут, что `'tbd'` обозначают оценки у которых недостаточно проголосовавших. То есть например игру оценили менее 4 игроков.

Заменим `'tbd'` на `13`, потомучто оценка не можеть быть больше **10**.

In [54]:
games_df['user_score'].replace(
    to_replace='tbd',
    value=13,
    inplace=True
)

Заменим `NaN` на `-1`, то есть значение пропущенно.

In [56]:
games_df['user_score'].fillna(
    value=-1,
    inplace=True
)

In [52]:
display(
    games_df['user_score'].describe()
)

count     10014
unique       96
top          13
freq       2424
Name: user_score, dtype: int64

In [20]:
display(
    games_df.loc[
        games_df['user_score'] != 'tbd',
            'user_score'
    ].describe()
)

count     7590
unique      95
top        7.8
freq       324
Name: user_score, dtype: object

#### `rating` — рейтинг от организации ESRB

(англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

In [36]:
display(
    games_df.loc[
        games_df['rating'].isna(),
        'name'
    ].count()
)

6764

In [23]:
display(
    games_df.loc[
        games_df['rating'].notna(),
            'rating'
    ].unique()
)

array(['E', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

- ` E ` — Everyone → 1994–1998 (as K-A), 1998–present (as E)
- ` M ` — Mature 17+ → 1994–present
- ` T ` — Teen → 1994–present
- ` E10+ ` — Everyone 10+ → 2005–present
- ` K-A ` — Everyone → 1994–1998
- ` A-0 ` — Adults Only 18+ → 1994–present
- ` EC ` — Early Childhood → 1994–2018
- ` RP ` — Rating Pending → 1994–present
- ` no_ESRB ` — without ESRB

In [49]:
games_df['rating'].fillna(
    value='no_ESRB',
    inplace=True
)

In [48]:
display(
    games_df.loc[
        games_df['rating'].isna()
        , 'name'
    ].count()
)

0

✅️ ‘name’ `NaN`
⭕️ 'year_of_release' `NaN`
✅️ 'genre' `NaN`
⭕️ 'critic_score' `NaN`
⭕️ 'user_score' `NaN`
✅️ 'rating' `NaN`

#### `na_sales` — продажи в Северной Америке (миллионы проданных копий)

In [22]:
display(
    games_df['na_sales'].describe(),
    games_df.loc[games_df['na_sales'] > 4]
)

count    16713.000000
mean         0.263302
std          0.813565
min          0.000000
25%          0.000000
50%          0.080000
75%          0.240000
max         41.360000
Name: na_sales, dtype: float64

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
145,Sonic the Hedgehog 2,GEN,1992.0,Platform,4.47,1.20,0.16,0.19,NaN,NaN,NaN
183,Madden NFL 2004,PS2,NaN,Sports,4.26,0.26,0.01,0.71,94.0,8.5,E
238,Madden NFL 2005,PS2,2004.0,Sports,4.18,0.26,0.01,0.08,91.0,7.9,E
240,Pitfall!,2600,1981.0,Platform,4.21,0.24,0.00,0.05,NaN,NaN,NaN


#### `eu_sales` — продажи в Европе (миллионы проданных копий)

In [23]:
display(
    games_df['eu_sales'].describe()
)

count    16713.000000
mean         0.145045
std          0.503359
min          0.000000
25%          0.000000
50%          0.020000
75%          0.110000
max         28.960000
Name: eu_sales, dtype: float64

#### `jp_sales` — продажи в Японии (миллионы проданных копий)

In [24]:
display(
    games_df['jp_sales'].describe()
)

count    16713.000000
mean         0.077625
std          0.308871
min          0.000000
25%          0.000000
50%          0.000000
75%          0.040000
max         10.220000
Name: jp_sales, dtype: float64

#### `other_sales` — продажи в других странах (миллионы проданных копий)

In [25]:
display(
    games_df['other_sales'].describe()
)

count    16713.000000
mean         0.047343
std          0.186742
min          0.000000
25%          0.000000
50%          0.010000
75%          0.030000
max         10.570000
Name: other_sales, dtype: float64

<hr>

## Шаг 2. Подготовьте данные

In [26]:
display(
    games_df.columns
)

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;

In [ ]:
games_df = games_df.astype(
    {
        # 'name': '',
        'platform': 'category',
        'year_of_release': 'unit16',
        'genre': 'category',
        'na_sales': '',
        'eu_sales': '',
        'jp_sales': '',
        'other_sales': '',
        'critic_score': '',
        'user_score': 'float16',
        'rating': 'category',
    }
)

Обработайте пропуски при необходимости:
- Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
- Опишите причины, которые могли привести к пропускам;

Обратите внимание на аббревиатуру 'tbd' в столбце с оценкой пользователей. Отдельно разберите это значение и опишите, как его обработать;

Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.

<hr>

## Шаг 3. Проведите исследовательский анализ данных

<hr>

## Шаг 4. Составьте портрет пользователя каждого региона

<hr>

## Шаг 5. Проверьте гипотезы

<hr>

## Шаг 6. Напишите общий вывод